# Tensorflow with Estimators

As we might have seen many times how to build a full Multi-Layer Perceptron model with full Sessions in Tensorflow. Unfortunately this was an extremely involved process. However developers have created Estimators that have an easier to use flow!

It is much easier to use, but you sacrifice some level of customization of your model. Let's go ahead and explore it!

## About the Data

Iris data set.
Based on some given features we need to predict the class of the flower(From the 3 classes of flowers).
Let's get the data:

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('Desktop/Tf estimator/iris.csv')

In [29]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [30]:
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']

In [31]:
X = df.drop('target',axis=1)
y = df['target'].apply(int)

## Train Test Split

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Estimators

Let's show you how to use the simpler Estimator interface!

In [34]:
import tensorflow as tf

## Feature Columns

In [35]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [36]:
feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [37]:
feat_cols

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Input Function

In [38]:
# there is also a pandas_input_fn we'll see in the exercise!!
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=5,shuffle=True)

In [39]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/tl/p86t_hm90_3_mlxscm7n7htw0000gn/T/tmp4tvt4oar', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11dc7c080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [40]:
classifier.train(input_fn=input_func,steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/tl/p86t_hm90_3_mlxscm7n7htw0000gn/T/tmp4tvt4oar/model.ckpt.
INFO:tensorflow:loss = 10.874613, step = 1
INFO:tensorflow:Saving checkpoints for 50 into /var/folders/tl/p86t_hm90_3_mlxscm7n7htw0000gn/T/tmp4tvt4oar/model.ckpt.
INFO:tensorflow:Loss for final step: 2.1692276.


## Model Evaluation

** Use the predict method from the classifier model to create predictions from X_test **

In [41]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [42]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/tl/p86t_hm90_3_mlxscm7n7htw0000gn/T/tmp4tvt4oar/model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [43]:
note_predictions[0]

{'logits': array([-1.8922976,  2.0086737,  2.4141536], dtype=float32),
 'probabilities': array([0.00802393, 0.3967869 , 0.59518915], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'2'], dtype=object)}

In [44]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

** Now create a classification report and a Confusion Matrix. Does anything stand out to you?**

In [45]:
from sklearn.metrics import classification_report,confusion_matrix

In [46]:
print(confusion_matrix(y_test,final_preds))

[[16  0  0]
 [ 0  9  6]
 [ 0  0 14]]


In [47]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        16
          1       1.00      0.60      0.75        15
          2       0.70      1.00      0.82        14

avg / total       0.91      0.87      0.86        45

